In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report


### Load dataset

In [3]:
true_news = pd.read_csv('True.csv/True.csv')
fake_news = pd.read_csv('Fake.csv/Fake.csv')

In [4]:
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
## Add label to the data
true_news['label'] = 1
fake_news['label'] = 0

In [7]:
## Combine both dataset
df = pd.concat([true_news, fake_news], axis=0).reset_index(drop=True)

In [8]:
## Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,title,text,subject,date,label
0,Fighting between rebel and army kills 27 in So...,Juba (Reuters) - Twenty-seven people were kill...,worldnews,"November 23, 2017",1
1,U.S. Sen. Warren predicts appeal in legal batt...,WASHINGTON (Reuters) - U.S. Senator Elizabeth ...,politicsNews,"November 28, 2017",1
2,Wake Up America! “SEED COMMUNITIES” Of Muslim ...,Be aware and be ready to push back on this eff...,Government News,"Apr 9, 2015",0
3,MESSAGE TO THE LEFT ON IMMIGRATION: If You Dis...,IT S A MUST READ! The first month of the Trump...,politics,"Feb 27, 2017",0
4,Japan's Abe says North Korea situation needs q...,MOSCOW (Reuters) - The crisis around North Kor...,worldnews,"September 7, 2017",1


## Text Processing

In [9]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Define these once, outside the function
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()  # Merge multiple spaces
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing
df['text'] = df['text'].apply(preprocess_text)


In [12]:
df.head()

,title,text,subject,date,label
0,Fighting between rebel and army kills 27 in So...,juba reuters twenty seven people killed rebel ...,worldnews,"November 23, 2017",1
1,U.S. Sen. Warren predicts appeal in legal batt...,washington reuters u senator elizabeth warren ...,politicsNews,"November 28, 2017",1
2,Wake Up America! “SEED COMMUNITIES” Of Muslim ...,aware ready push back effort state department ...,Government News,"Apr 9, 2015",0
3,MESSAGE TO THE LEFT ON IMMIGRATION: If You Dis...,must read first month trump administration alr...,politics,"Feb 27, 2017",0
4,Japan's Abe says North Korea situation needs q...,moscow reuters crisis around north korea requi...,worldnews,"September 7, 2017",1


## Train-Test Split

In [29]:
from sklearn.model_selection import train_test_split
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
y.shape

(44898,)

### Vectorize Text Using TF-IDF

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit the vectorizer on the training data and transform both train and test data
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()
# Check the shape of the vectorized data
print(X_train_tfidf.shape, X_test_tfidf.shape)

(35918, 5000) (8980, 5000)


### Using Word2Vec

In [33]:
from sklearn.linear_model import LogisticRegression

# Train the model on the training data
model = LogisticRegression()
model.fit(X_train_tfidf,y_train)

# Predict on the test data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
print(accuracy_score(y_pred,y_test))
print()
print(classification_report(y_test,y_pred))

0.9869710467706013

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4660
           1       0.98      0.99      0.99      4320

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

